# query execution plans

In [15]:
# reset the environment
!chmod +x ./reset.sh && ./reset.sh >> /dev/null

./reset.sh: line 10: return: can only `return' from a function or sourced script
Did not find any relations.
ERROR:  cannot drop the currently open database
ERROR:  current user cannot be dropped


In [18]:
# show tables
!psql postgres -c "\dt+"

                                      List of relations
 Schema |    Name     | Type  |  Owner  | Persistence | Access method |  Size   | Description 
--------+-------------+-------+---------+-------------+---------------+---------+-------------
 public | badges      | table | sueszli | permanent   | heap          | 2144 kB | 
 public | comments    | table | sueszli | permanent   | heap          | 8288 kB | 
 public | posthistory | table | sueszli | permanent   | heap          | 31 MB   | 
 public | postlinks   | table | sueszli | permanent   | heap          | 152 kB  | 
 public | posts       | table | sueszli | permanent   | heap          | 15 MB   | 
 public | r           | table | sueszli | permanent   | heap          | 1024 kB | 
 public | s           | table | sueszli | permanent   | heap          | 128 kB  | 
 public | t           | table | sueszli | permanent   | heap          | 1024 kB | 
 public | tags        | table | sueszli | permanent   | heap          | 56 kB   | 
 public

In [19]:
# show schema of r, s, t tables
!psql postgres -c "\d r"
!psql postgres -c "\d s"
!psql postgres -c "\d t"

                 Table "public.r"
 Column |  Type   | Collation | Nullable | Default 
--------+---------+-----------+----------+---------
 a      | integer |           |          | 
 b      | integer |           |          | 

                 Table "public.s"
 Column |  Type   | Collation | Nullable | Default 
--------+---------+-----------+----------+---------
 b      | integer |           |          | 
 c      | integer |           |          | 

                 Table "public.t"
 Column |  Type   | Collation | Nullable | Default 
--------+---------+-----------+----------+---------
 a      | integer |           |          | 
 c      | integer |           |          | 



let's look at the triangle join $R \bowtie S \bowtie T$.

